In [1]:
# encoding:utf-8
from tkinter import * 
import cv2
import tensorflow as tf
from PIL import Image, ImageTk
from tkinter import filedialog
from tkinter.filedialog import askopenfile
import sys
import time
import logging
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
import matplotlib.pyplot as plt
from matplotlib import image
import numpy as np
from skimage import data , color
from skimage.transform import rescale , resize , downscale_local_mean
import os
import io
sys.path.append('d:\\anaconda\\lib\\site-packages')
import pyodbc
import hashlib

db = pyodbc.connect(
    'Driver={SQL Server};'
    'Server=DESKTOP-G5GUT4H;'
    'Database=NTU;'
    'Trusted_Connection=True;'
)
db_araci = db.cursor()




classes = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]
model = tf.keras.models.load_model("C:/Users/Kayra/OneDrive/Masaüstü/Nesne Tespit Modeller/InceptionV3Yeni.h5")

master = Tk()
canvas = Canvas(master, height = 750 , width = 1150)
canvas.pack()
frame_ust = Frame(master , bg='#add8e6')
frame_ust.place(relx= 0.05 , rely= 0.05 , relwidth=0.9 , relheight=0.15)

frame_orta = Frame(master)
frame_orta.place(relx=0.05, rely=0.2, relwidth=0.9, relheight=0.65)

frame_alt = Frame(master, bg='#add8e6')
frame_alt.place(relx = 0.05 , rely = 0.85 , relwidth = 0.9, relheight = 0.1)



sinif_göster = ""

nesne_sinifi_etiketi = Label(frame_alt, bg='#add8e6', text = sinif_göster)


with open('dosyaYolu.txt', 'r', encoding="utf-8", errors='ignore') as f:
    path = f.read()
    


def resim_tespit(path):
    i1 = cv2.imread(path)
    newi1 = resize(i1,(32,32,3),)

    im = np.expand_dims(newi1 , axis = 0)
    keypoints = model.predict(im)
    sinif = classes[np.argmax(keypoints)]
    return sinif



def resim_tikla(event, id, image_data):
        
    popup = Toplevel()
    popup.geometry("800x600")
    image = Image.open(io.BytesIO(image_data)) 
    photo = ImageTk.PhotoImage(image)
    label = Label(popup, image=photo)
    label.image = photo
    label.pack()
    
    db_araci.execute("SELECT class[0] FROM image_inf WHERE id = ?", (id,))
    resim_sinif = db_araci.fetchone()[0]
    frame_popup_alt = Frame(popup, bg='#add8e6')
    frame_popup_alt.place(relx = 0.05 , rely = 0.85 , relwidth = 0.9, relheight = 0.1)

    result_label = Label(frame_popup_alt, text="a")
    result_label.config(text= resim_sinif)
    result_label.pack()
    

    

def on_created(event):
    eklenen_resim_yolu = event.src_path
    duzelmis_resim_yolu = eklenen_resim_yolu.replace("\\" , "/")
    
    #uzanti değişkeni dosyanın uzantısını alır (.jpg, .jpeg)
    uzanti = os.path.splitext(duzelmis_resim_yolu)[-1].lower()
    image_eklenen = Image.open(duzelmis_resim_yolu)
    data_eklenen = os.path.getsize(duzelmis_resim_yolu)
    
    with open(duzelmis_resim_yolu, "rb") as image_file:
        image_data = image_file.read()
        md5_hash = hashlib.md5(image_data).hexdigest()
    
    db_araci.execute("SELECT COUNT(*) FROM image_inf WHERE md5 = ?",(md5_hash))
    result = db_araci.fetchone()
    
    if result[0]  ==  0 :
    
        if (uzanti == ".jpg"):
            file_name = os.path.basename(event.src_path)
            db_araci.execute("insert into image_inf values(?, ?, ?, ?)", 
                             ( file_name,resim_tespit(duzelmis_resim_yolu),image_data,md5_hash))
            db_araci.commit()
        elif (uzanti == ".jpeg"):
            file_name = os.path.basename(event.src_path)
            db_araci.execute("insert into image_inf values(?, ?, ?, ?)",
                             ( file_name,resim_tespit(duzelmis_resim_yolu),image_data,md5_hash))
            db_araci.commit()
        elif (uzanti == ".png"):
            file_name = os.path.basename(event.src_path)
            db_araci.execute("insert into image_inf values(?, ?, ?, ?)",
                             ( file_name,resim_tespit(duzelmis_resim_yolu),image_data,md5_hash))
            db_araci.commit()
            

event_handler = FileSystemEventHandler()

event_handler.on_created = on_created

observer = Observer()
observer.schedule(event_handler, path, recursive=True)
observer.start()


def fotografSec():


    f_types = [( 'Jpeg Files', '*.jpeg')]
    filename = filedialog.askopenfilename(filetypes = (("jpeg files","*.jpg"),("adres_yolu","*.*")))
    sinif_göster = resim_tespit(filename)
    nesne_sinifi_etiketi.configure(text = sinif_göster)

    nesne_sinifi_etiketi.pack()
    

    
    image1 = Image.open(filename)
    img_resized=image1.resize((400,400))
    test = ImageTk.PhotoImage(img_resized)
    label1 = Label(image=test)
    label1.image = test
    label1.place(relx = 0.05 , rely = 0.25 , relwidth = 0.9 , relheight =0.55)
   
    return


def ayarlar():
    adres_yolu = filedialog.askdirectory()
    
    with open("dosyaYolu.txt", "w", encoding="utf-8") as dosya:
        dosya.write(
            '{}'.format(
                adres_yolu
            ))
        dosya.close
    return

def resim_tespit_tikla(photo):
        img = Image.open(io.BytesIO(photo))
        img_array = np.array(img)
        newPhoto = resize(img_array,(32,32,3),)
        im = np.expand_dims(newPhoto , axis = 0)
        keypoints = model.predict(im)
        sinif = classes[np.argmax(keypoints)]
        return sinif



# Resim verilerini veritabanından okuyun 

# Her resim verisi için önizleme oluşturun ve görüntüleyin
def önizleme():
    for i, row in enumerate(db_araci.execute("Select  id, images from image_inf")):
        id = row[0] # id değerini al
        photos=[]
        image_data = row[1] # resim verisini al
        image = Image.open(io.BytesIO(image_data))
        image.thumbnail((147, 147))
        photo = ImageTk.PhotoImage(image)
        photos.append(photo)
        label = Label(frame_orta, image=photos[-1])
        label.photo = photos[-1]
        label.grid(row=i // 6, column=i % 6, padx=5, pady=5)
        label.bind("<Double-Button-1>", lambda event, id=id, image_data=image_data: resim_tikla(event, id, image_data))


button_fotograf_sec = Button(frame_ust, text= 'Fotoğraf Seç', height = 5 , width = 15,  command = fotografSec)
button_fotograf_sec.pack(padx= 10 , pady= 10, side = LEFT)

button_yenile = Button(frame_ust, text= 'Yenile', height = 5 , width = 15, command = önizleme)
button_yenile.pack(padx= 10 , pady= 10, side = LEFT)

button_ayarlar = Button(frame_ust , text= 'Ayarlar', height = 5 , width = 15, command = ayarlar)
button_ayarlar.pack(padx= 10, pady= 10, side= RIGHT)

önizleme()
master.mainloop()
observer.stop()
observer.join()
db.close()
